In [9]:
from dotenv import load_dotenv
import requests
import json
import pandas as pd

In [10]:
# 動画IDの設定
videoId = 'wKk2op7zn8A'

# APIキーの取得
load_dotenv('.env')
key = os.getenv('API_KEY')

# URLの設定
URL = 'https://www.googleapis.com/youtube/v3/'

# 親コメントと子コメントを格納するリストの初期化
parent_comments = list()
child_comments = list()

In [11]:
# 親コメントを取得するための関数
def get_comment(comment_num, videoId, page_token):
  # パラメータの設定
  params = {
    'key': key,
    'part': 'snippet',
    'videoId': videoId,
    'maxResults': 100,
    'textFormat': 'plaintext',
  }

  # 何ページ目かを設定
  if page_token is not None:
    params['pageToken'] = page_token

  # リクエスト＆レスポンス
  response = requests.get(URL + 'commentThreads', params=params)
  resource = response.json()

  # 各コメントの情報を取得する
  for item in resource['items']:
    # 初期化
    parentId = user_name = reply_count = like_count = commented_at = comment = ''
    # 親コメントID
    parentId = str(item['snippet']['topLevelComment']['id'])
    # ユーザー名
    user_name = str(item['snippet']['topLevelComment']['snippet']['authorDisplayName'])
    # 返信数
    reply_count = int(item['snippet']['totalReplyCount'])
    # グッド数
    like_count = int(item['snippet']['topLevelComment']['snippet']['likeCount'])
    # コメント日時
    commented_at = item['snippet']['topLevelComment']['snippet']['publishedAt']
    # コメント
    comment = str(item['snippet']['topLevelComment']['snippet']['textDisplay'])

    # リストに追加
    parent_comments.append([videoId, parentId, user_name, reply_count, like_count, commented_at, comment])

    # 返信が存在する場合は取得する
    if reply_count >= 1:
      get_reply(1, videoId, parentId, None)

    comment_num += 1

  # 次のページが存在する場合は取得する
  if 'nextPageToken' in resource:
    get_comment(comment_num, videoId, resource["nextPageToken"])

In [12]:
# 子コメントを取得するための関数
def get_reply(reply_num, videoId, parentId, page_token):
  # パラメータの設定
  params = {
    'key': key,
    'part': 'snippet',
    'videoId': videoId,
    'parentId': parentId,
    'maxResults': 100,
    'textFormat': 'plaintext',
  }

  # 何ページ目かを設定
  if page_token is not None:
    params['pageToken'] = page_token

  # リクエスト＆レスポンス
  response = requests.get(URL + 'comments', params=params)
  resource = response.json()

  # 各返信の情報を取得する
  for item in resource['items']:
    # 初期化
    user_name = like_count = commented_at = comment = ''
    # ユーザー名
    user_name = str(item['snippet']['authorDisplayName'])
    # グッド数
    like_count = int(item['snippet']['likeCount'])
    # コメント日時
    commented_at = item['snippet']['publishedAt']
    # コメント
    comment = str(item['snippet']['textDisplay'])

    # リストに追加
    child_comments.append([videoId, parentId, user_name, like_count, commented_at, comment])

    reply_num += 1

  # 次のページが存在する場合は取得する
  if 'nextPageToken' in resource:
    get_reply(reply_num, videoId, parentId, resource["nextPageToken"])

In [13]:
# 全コメントの取得
get_comment(1, videoId, None)

In [14]:
# リストをデータフレーム化
parent_comments_df = pd.DataFrame(parent_comments, columns=['動画ID', '親コメントID', 'ユーザー名', '返信数', 'グッド数', 'コメント日時', 'コメント'])
child_comments_df = pd.DataFrame(child_comments, columns=['動画ID', '親コメントID', 'ユーザー名', 'グッド数', 'コメント日時', 'コメント'])

In [15]:
parent_comments_df

,動画ID,親コメントID,ユーザー名,返信数,グッド数,コメント日時,コメント
0,wKk2op7zn8A,UgwM9fFrJAFPDG_lBhR4AaABAg,桜井ポンスケ,0,0,2023-01-04T07:14:02Z,前に会社の人が、雪かき嫌で引っ越して来たって、言ってたけな……\nこの時期になると思い出す。...
1,wKk2op7zn8A,Ugwxj5UDhj3eDLzbtJR4AaABAg,Hakoniwa Train / 東京都立川市,0,0,2023-01-02T03:56:41Z,🦊「雪はきれいだよね。雪が降る場所に移住しようかな」と、大雪を経験したことがない南国育ちの人...
2,wKk2op7zn8A,Ugz-76bRENAt59XgmO94AaABAg,滅,0,1,2022-12-31T03:25:25Z,こっちなんてめっちゃ晴れてるのに
3,wKk2op7zn8A,UgxQTZgKxNpKPLDpJ5d4AaABAg,フロッピーグレース,0,0,2022-12-30T08:44:27Z,凄い雪ですね。(^_^;)
4,wKk2op7zn8A,Ugyx5WYXVjWPT_89jzZ4AaABAg,フクロウパイ,0,2,2022-12-30T07:22:08Z,ふ〜ん、温暖化の影響って報道しないのはおかしいな💢
...,...,...,...,...,...,...,...
295,wKk2op7zn8A,Ugz42PRH_WSAuuEkvn94AaABAg,ラミン222,2,11,2022-12-24T03:06:55Z,ノーマルタイヤで行こうとしないでくれちょっとは考えたらいいのに
296,wKk2op7zn8A,Ugzg_Ih8QcYbyGEo5J14AaABAg,シャナオウ 中村,0,2,2022-12-24T03:06:05Z,三重県津市じゃなくて良かった
297,wKk2op7zn8A,UgzcFp5JV3LohzgAkpl4AaABAg,Nash Huang,0,46,2022-12-24T03:04:51Z,請做好保暖~
298,wKk2op7zn8A,UgyQLE6R56ZzdsqtFPp4AaABAg,CAFE LATTE,7,8,2022-12-24T02:55:48Z,凄く無くない、普通じゃね？


In [16]:
child_comments_df

,動画ID,親コメントID,ユーザー名,グッド数,コメント日時,コメント
0,wKk2op7zn8A,UgzQbBIz5wnThEB4XQF4AaABAg,Carrot【Kochi】【Country vs Country】,0,2023-03-17T14:19:28Z,黙れ都会ぶってる国民
1,wKk2op7zn8A,Ugyb7JSWlABvzkALV9l4AaABAg,ぷりん,3,2022-12-28T08:18:41Z,@Kenichi Tomizawa それですね☺\n上辺だけしかみない人がいかに多いかがよく...
2,wKk2op7zn8A,Ugyb7JSWlABvzkALV9l4AaABAg,Kenichi Tomizawa,4,2022-12-28T08:08:41Z,@ぷりん \nSDGSもですね↗️
3,wKk2op7zn8A,Ugyb7JSWlABvzkALV9l4AaABAg,ぷりん,3,2022-12-28T07:53:14Z,@Kenichi Tomizawa ECOはお金になりますからね\n国や企業がECOだのリサ...
4,wKk2op7zn8A,Ugyb7JSWlABvzkALV9l4AaABAg,Kenichi Tomizawa,3,2022-12-28T01:32:09Z,@takezo yamada \nそうですねｗ\n寒くても暑くても、地球温暖化＆気候変動（最...
...,...,...,...,...,...,...
140,wKk2op7zn8A,UgyQLE6R56ZzdsqtFPp4AaABAg,Melvin Manhoef,1,2022-12-24T03:50:29Z,うん。凄くなくない
141,wKk2op7zn8A,UgyQLE6R56ZzdsqtFPp4AaABAg,ug1051,0,2022-12-24T03:35:31Z,@ムーランシュー様 ムーシュー様々！
142,wKk2op7zn8A,UgyQLE6R56ZzdsqtFPp4AaABAg,kara mo,0,2022-12-24T03:27:07Z,雪は毎年降らない地域なのかな？
143,wKk2op7zn8A,UgyQLE6R56ZzdsqtFPp4AaABAg,ug1051,5,2022-12-24T03:09:26Z,普通だね。冬だもん


In [17]:
# CSV形式で出力
parent_comments_df.to_csv(f"parent_{videoId}.csv", index=False, encoding='utf-8')
child_comments_df.to_csv(f"child_{videoId}.csv", index=False, encoding='utf-8')